В этом ноутбуке я буду делать функцию, которая будет преобразовывать список, который мы получим после вызова функции client.get_klines() в формат, для входа в нейросеть

In [1]:
from binance.client import Client
from config import api_key,secret_key

asset = "BTCUSDT"
client = Client(api_key = api_key, api_secret = secret_key)
test1 = client.get_klines(symbol = asset, interval = "1m", limit = 15)
print(test1)

[[1612819380000, '44219.81000000', '44285.92000000', '44207.47000000', '44280.08000000', '62.32115100', 1612819439999, '2758361.62060988', 2472, '34.77216400', '1539066.67621088', '0'], [1612819440000, '44280.07000000', '44294.80000000', '44206.96000000', '44241.48000000', '103.71947000', 1612819499999, '4589903.35490830', 2625, '32.44526600', '1435678.05324685', '0'], [1612819500000, '44240.20000000', '44250.00000000', '44148.00000000', '44164.31000000', '89.54441300', 1612819559999, '3958465.40516676', 2551, '23.85320900', '1054672.39239437', '0'], [1612819560000, '44164.31000000', '44238.37000000', '44119.95000000', '44188.91000000', '75.39729300', 1612819619999, '3330130.76745657', 2511, '27.56871500', '1217701.55174998', '0'], [1612819620000, '44188.91000000', '44265.29000000', '44165.81000000', '44240.90000000', '72.10208600', 1612819679999, '3186372.17244632', 2642, '34.97574700', '1545715.15843619', '0'], [1612819680000, '44240.91000000', '44400.00000000', '44206.54000000', '44

Видим, что есть лишние столбцы, форматы строк, а не чисел, а также отсутствие столбцов месяца,дня и тд. Также, надо отмаштабировать

Соответственно, надо сделать следующее:

    1. Добавить столбцы месяца, дня, часа(+3), минуты
    
    2. Удалить столбцы, которых у нас нет в обучении
    
    3. Сделать тип данных float32
    
    4. Отмаштабировать, использую scaler, натренированный на обучающих данных

Сделаем все поэтапно для наглядности, а потом напишем резулльтирующую функцию

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time



df = pd.DataFrame(data = test1)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1612819380000,44219.81000000,44285.92000000,44207.47000000,44280.08000000,62.32115100,1612819439999,2758361.62060988,2472,34.77216400,1539066.67621088,0
1,1612819440000,44280.07000000,44294.80000000,44206.96000000,44241.48000000,103.71947000,1612819499999,4589903.35490830,2625,32.44526600,1435678.05324685,0
2,1612819500000,44240.20000000,44250.00000000,44148.00000000,44164.31000000,89.54441300,1612819559999,3958465.40516676,2551,23.85320900,1054672.39239437,0
3,1612819560000,44164.31000000,44238.37000000,44119.95000000,44188.91000000,75.39729300,1612819619999,3330130.76745657,2511,27.56871500,1217701.55174998,0
4,1612819620000,44188.91000000,44265.29000000,44165.81000000,44240.90000000,72.10208600,1612819679999,3186372.17244632,2642,34.97574700,1545715.15843619,0


In [4]:
#добавим месяцы, часы, дни и минуты
df["month"] = pd.to_datetime(df[0], unit = "ms").dt.month
df["day"] = pd.to_datetime(df[0], unit = "ms").dt.day
df["hour"] = pd.to_datetime(df[0], unit = "ms").dt.hour + 3
df["minute"] = pd.to_datetime(df[0], unit = "ms").dt.minute

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,month,day,hour,minute
0,1612819380000,44219.81000000,44285.92000000,44207.47000000,44280.08000000,62.32115100,1612819439999,2758361.62060988,2472,34.77216400,1539066.67621088,0,2,8,24,23
1,1612819440000,44280.07000000,44294.80000000,44206.96000000,44241.48000000,103.71947000,1612819499999,4589903.35490830,2625,32.44526600,1435678.05324685,0,2,8,24,24
2,1612819500000,44240.20000000,44250.00000000,44148.00000000,44164.31000000,89.54441300,1612819559999,3958465.40516676,2551,23.85320900,1054672.39239437,0,2,8,24,25
3,1612819560000,44164.31000000,44238.37000000,44119.95000000,44188.91000000,75.39729300,1612819619999,3330130.76745657,2511,27.56871500,1217701.55174998,0,2,8,24,26
4,1612819620000,44188.91000000,44265.29000000,44165.81000000,44240.90000000,72.10208600,1612819679999,3186372.17244632,2642,34.97574700,1545715.15843619,0,2,8,24,27


In [6]:
#удалям ненужные столбцы
df = df.drop(columns = [0,6,11])

In [8]:
#преобразуем тип данных
df = df.astype("float32")

In [12]:
#обчаем scaler на обучающих данных
df_to_fit_scaler = pd.read_csv("df_to_fit_scaler.csv")
scaler = MinMaxScaler()
scaler.fit(df_to_fit_scaler)

MinMaxScaler()

In [13]:
df_to_fit_scaler.head()

,1,2,3,4,5,7,8,9,10,month,day,hour,minute
0,11322.05,11323.02,11294.00,11297.15,75.471279,8.534482e+05,1200,24.872009,281269.304392,7,31,21,1
1,11298.56,11332.02,11297.07,11330.86,57.373999,6.492224e+05,1486,36.822923,416647.500196,7,31,21,2
2,11331.09,11350.06,11330.86,11347.44,76.438368,8.670462e+05,1114,59.610735,676161.703455,7,31,21,3
3,11347.44,11365.28,11347.44,11363.27,90.757347,1.030781e+06,1130,52.535249,596693.051054,7,31,21,4
4,11363.46,11365.52,11363.27,11364.08,21.991909,2.499153e+05,332,4.848286,55096.556460,7,31,21,5


In [14]:
scal = scaler.transform(df)

In [15]:
scal.shape

(15, 13)

вот, получили необходимый формат для входа в нейросеть

теперь можем объединить все и написать итоговую функцию

In [16]:
#функция будет принимать на вход список из 15 данных об последних минутах, а также обученный объект для маштабирования
def convert(array, scaler):
    df = pd.DataFrame(data = array)
    #добавляем данные из timestamp
    df["month"] = pd.to_datetime(df[0], unit = "ms").dt.month
    df["day"] = pd.to_datetime(df[0], unit = "ms").dt.day
    df["hour"] = pd.to_datetime(df[0], unit = "ms").dt.hour + 3
    df["minute"] = pd.to_datetime(df[0], unit = "ms").dt.minute
    #удаляем ненужные столбцы
    df = df.drop(columns = [0,6,11])
    #преобразуем в нужный тип данных
    df = df.astype("float32")
    #маштабируем
    result = scaler.transform(df)
    return result

In [17]:
#Посмотрим, сколько времени будет выполняться наша преобразование к нужному формату
start_time = time.time()
print(convert(test1,scaler))
print(time.time() - start_time)

[[1.0726225  1.0728744  1.0752347  1.0745031  0.02726522 0.04833274
  0.10851148 0.01795891 0.04027944 0.09090909 0.23333335 1.0434783
  0.3898305 ]
 [1.074505   1.0731515  1.0752188  1.0732974  0.0453768  0.0804255
  0.1152276  0.01675713 0.03757362 0.09090909 0.23333335 1.0434783
  0.40677965]
 [1.0732595  1.0717539  1.073375   1.0708867  0.03917528 0.06936128
  0.11197928 0.01231956 0.02760219 0.09090909 0.23333335 1.0434783
  0.42372882]
 [1.0708888  1.071391   1.0724977  1.0716553  0.03298598 0.05835143
  0.11022343 0.01423852 0.03186888 0.09090909 0.23333335 1.0434783
  0.44067797]
 [1.0716573  1.0722308  1.0739318  1.0732791  0.03154434 0.05583245
  0.11597384 0.01806406 0.04045344 0.09090909 0.23333335 1.0434783
  0.45762712]
 [1.0732816  1.0764334  1.0752056  1.0779129  0.06696386 0.11886833
  0.18313506 0.0503961  0.11321548 0.09090909 0.23333335 1.0434783
  0.47457626]
 [1.0779256  1.0764334  1.0788883  1.078185   0.03438739 0.06109741
  0.12527984 0.01783524 0.04009598 0.09